In [1]:
import glob
import sqlite3
from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd

In [2]:
BCU_database_folder = '../data/BCU_database/'

Extract the Photo table from the database

In [3]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(BCU_database_folder + 'bcu_caption.db')
Photo = pd.read_sql_query("SELECT * from Photo", con)

# Verify that result of SQL query is stored in the dataframe
print('original number of picture :',len(Photo))
Photo

original number of picture : 2216


,id,bcu_id,title,title_en,caption,year_from,year_to,photographer_name,fund_name,locality,theme,filename,title_en_preprocessed,caption_preprocessed
0,1,ALBL00222,"Singine, Sense, Düdingen, moulin","Singine, Sense, Düdingen, mill",aerial view of the town.,1952,1952,None,Aéroport Lausanne Blécherette,Düdingen,Question de la modernité - urbanisme tardif,ALBL00222_2k_324w.jpg,"singine, sense, düdingen, mill",None
1,2,ALCU_00005,Inauguration de la plaque commémorative au mur...,Inauguration of the commemorative plaque on th...,crowds gather to watch the parade.,1900,1900,"Cuony, Xavier","Cuony, Xavier et Albert",Fribourg,Question religieuse - formation,ALCU_00005_2k_324w.jpg,inauguration of the commemorative plaque on th...,crowds gather to watch the parade.
2,3,ALCU_00009,"Marché à la Place Notre-Dame, autour de la fon...","Marché à la Place Notre-Dame, autour de la fou...","a city in the past, history.",1900,1900,"Cuony, Xavier","Cuony, Xavier et Albert",Fribourg,Question de la modernité - alimentation,ALCU_00009_2k_324w.jpg,"marché à la place notre-dame, autour de la fou...",None
3,4,ALCU_00033,"Patient du Dr Xavier Cuony, physicien de ville...","Patient of Dr Xavier Cuony, city physicist, Fr...",painting artist in his studio.,1895,1910,"Cuony, Xavier","Cuony, Xavier et Albert",Fribourg,Question de la modernité - santé,ALCU_00033_2k_324w.jpg,"patient of dr xavier cuony, city physicist,",painting artist in his studio.
4,5,ALNU_00015,"Homme âgé (Aloïs Catillaz), Felsenegg, Wünnewil","Elderly (Aloïs Catillaz), Felsenegg, Wünnewil",portrait of a man sitting on a bench.,1885,1900,"Nussbaumer, Aloïs","Nussbaumer, Aloïs",Wünnewil,Question sociale,ALNU_00015_2k_324w.jpg,"elderly (aloïs catillaz), felsenegg, wünnewil",a man sitting on a bench.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211,2212,VIBU_00758,Ouvriers dans les ateliers de la nouvelle tuil...,Workers in the workshops of the new tile facto...,photograph of a group of men working on a mach...,1932,1932,"Buchs, Victor","Buchs, Victor",None,Question de la modernité - tension ville-campagne,VIBU_00758_2k_324w.jpg,workers in the workshops of the new tile facto...,a group of men working on a machine.
2212,2213,VIBU_00800,Funérailles du conseiller d'Etat Georges Pytho...,Funeral of the State Councillor Georges Python...,photograph of politician walking down the street.,1927,1927,"Buchs, Victor","Buchs, Victor",Fribourg,Question politique - droite conservatrice,VIBU_00800_2k_324w.jpg,funeral of the state councillor georges python...,politician walking down the street.
2213,2214,VIBU_00823,Femmes détenues au pénitentier de Bellechasse ...,Women held at Bellechasse Penitentiary for the...,photograph of a group of people working in a f...,1930,1933,"Buchs, Victor","Buchs, Victor",Bellechasse,Question sociale - justice-criminalité,VIBU_00823_2k_324w.jpg,women held at bellechasse penitentiary for the...,a group of people working in a field.
2214,2215,aueg_00004,None,,None,None,None,None,None,None,Question de genre - représentations du féminin,AUEG_00004_2k_324w.jpg,,None


Get all the images

In [10]:
mypath = BCU_database_folder + '03_resized'
onlyfiles = [f for f in listdir() if isfile(join(mypath, f))]

In [12]:
files = glob.glob(mypath + '/*')

In [13]:
files_324w = list(filter(lambda k: '324w' in k, files))

In [14]:
missing = []
Photo['filename'] = Photo.loc[~Photo.bcu_id.str.contains('BCUF')].bcu_id.apply(lambda x:  (list(filter(lambda k: x.lower() in k.lower(), files_324w)))[0].removeprefix(mypath + '/') if len(list(filter(lambda k: x.lower() in k.lower(), files_324w))) > 0 else missing.append(x))
print(f'Number of pictures with filenames: {Photo.filename.notnull().sum()}')

Number of pictures with filenames: 1908


Processing titles and captions

In [15]:
Photo['title_en_preprocessed'] = Photo["title_en"].apply(lambda x: np.nan if str(x).find('Souvenir of') != -1 else str(x).lower().replace('portrait of ', '').replace('canton of fribourg','').replace('(fribourg)','').replace('[fribourg]','').replace('of fribourg','').replace('fribourg',''))
Photo['caption_preprocessed'] = Photo["caption"].apply(lambda x: np.nan if (str(x).find('a city in') != -1 or str(x).find('view') != -1) else str(x).replace('portrait of ', '').replace('photograph of ', '').replace('black and white photo of ', ''))
print(f'Number of images with processed captions and filenames: {(Photo.caption_preprocessed.notnull() & Photo.filename.notnull()).sum()}')
print(f'Number of images with processed titles and filenames: {(Photo.title_en_preprocessed.notnull() & Photo.filename.notnull()).sum()}')

Number of images with processed captions and filenames: 1506
Number of images with processed titles and filenames: 1881


In [42]:
Photo.to_sql("Photo",con,if_exists='replace',index=False, index_label="id")

2216